## Pythonic setup

In [14]:
import numpy as np

% matplotlib inline
from seaborn import plt
plt.rcParams['figure.figsize'] = (10, 5)

## data

100 points generated from a mixture distribution of two normal distributions with R.


```R
N      <- 100
a_true <- 0.4
mean1  <- 0
mean2  <- 3
sd1    <- 1
sd2    <- 1
Y      <- c(rnorm((1-a_true)*N, mean1, sd1), rnorm(a_true*N, mean2, sd2))
data   <- list(N=N, Y=Y)

write.table(Y, file="points.csv", sep=",", row.names=F, col.names=F)
```

## model (model 2, single normal distribution)

We have point data yi,i=1,…,N. We wish to find the posterior distributions of the coefficients mu(the mean) and s(the standard deviation). The model can be written as

$$y_i \sim \mathcal{N}(\mu, \sigma^2)$$

The likelihood for this model may be written as the product over N iid observations

$$L(y_1, \ldots, y_N | \mu, \sigma) = \prod_{i = 1}^N \mathcal{N}(\mu, \sigma^2)$$

We place priors on mu and s.

$$\mu \sim uniform(-\infty, \infty) $$
$$\sigma \sim uniform(0, \infty) $$

and then $$ p(\mu)=constant$$ $$p(\sigma) = constant $$

### Update for mu

We are interested in finding

\begin{eqnarray}
p(\mu | \sigma, Y) &=& \frac {p(\mu, \sigma, Y)} {p(\sigma, Y)} \\
&\propto& p(Y | \mu, \sigma) p(\mu) p(\sigma) \\
&\propto& p(Y | \mu, \sigma) \\
\end{eqnarray}

Note that $p(Y | \mu, \sigma)$ is just the likelihood from above. 

If a variable x follows a normal distribution with mean $\mu$ and variance $\sigma^2$ then $ lp(x)$ (the log-dependence on x) is

$$ lp(x) \propto - \frac{1} {2\sigma^2} (x-\mu)^2 \propto - \frac{1} {2\sigma^2}x^2 +  \frac{\mu} {\sigma^2} x $$

So if we can force the log-posterior conditional density into a quadratic form then the coefficient of $x$ will be $ \frac{\mu} {\sigma^2}$ and the coefficient of $ x^2 $ will be $ - \frac{1} {2\sigma^2} $.

Hence $ lp(\mu) $ is 

\begin{eqnarray}
lp(\mu) &\propto& -  \frac {1} {2\sigma^2} \sum_{i=1}^N \left(y_i- \mu \right)^2 \\
&=&  -  \frac {1} {2\sigma^2} \sum_{i=1}^N \left(y_i^2 - 2y_i\mu + \mu^2 \right) \\
&\propto&  -  \frac {N} {2\sigma^2} \mu^2 + \left(\frac{1} {\sigma^2} \sum_{i=1}^N y_i\right)\mu \\
\end{eqnarray}

This expression is quadratic in $\mu$, meaning the conditional sampling density for $\mu$ will also be normal. The coefficient of $ \mu$ is $ \frac{1} {\sigma^2} \sum_{i=1}^N y_i $ while the coefficient of $ \mu^2 $ is $ -  \frac {N} {2\sigma^2}$. This implies the conditional sampling distribution of $ \mu$ is 

$$ \mu | \sigma, Y \sim \mathcal{N}\left( \frac{1} {N} \sum_{i=1}^N (y_i), \frac {\sigma^2} {N} \right)$$



In [16]:
def sample_mu(y, s):
    N = len(y)
    mean = np.sum(y) / N
    variance = s * s / N
    return np.random.normal(mean, np.sqrt(variance))

## Update for s


We are interested in finding

$$ p(\sigma | \mu, Y) \propto p(Y | \mu, \sigma) $$





In [10]:
from subprocess import check_output

print(check_output(["Rscript", "run.r"]).decode("utf8"))

model 2a
       mean      se_mean        sd      2.5%      25%      50%      75%    97.5%    n_eff     Rhat
mu 1.307905 0.0009655421 0.1756840 0.9634475 1.189366 1.308708 1.427303 1.652775 33107.17 1.000021
s  1.752572 0.0007169266 0.1265588 1.5272902 1.663740 1.744615 1.832487 2.024040 31162.72 1.000007

model 2b
       mean     se_mean        sd      2.5%      25%      50%      75%    97.5%    n_eff     Rhat
mu 1.310476 0.002452783 0.4077607 0.5089039 1.041285 1.310512 1.577748 2.117387 27637.11 1.000143
s  1.878565 0.001913403 0.3144175 1.3825179 1.656800 1.839398 2.058361 2.596761 27002.27 1.000107

           WAIC     WBIC
model2 1.980418 201.2776

